
# Examen 2 — Modelos de infecciones con umbral (SIR, SIRV, SIRVQ)
**Instrucciones generales**  
- Trabaja en este cuaderno, sin crear funciones fuera de las firmas provistas.  
- Valida parámetros (valores no negativos, tasas en [0,1] cuando corresponda).  
- Reporta métricas pedidas y acompaña cada simulación con **gráfica(s)** con títulos/leyendas/ejes.

**Qué entregar (por modelo)**  
1) Implementación correcta de las series de tiempo.  
2) Gráfica(s) de la serie temporal.  
3) Métricas solicitadas.  
4) Preguntas interpretando el resultado.



## Setup
Ejecuta esta celda primero.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

def graficar_trayectorias(series_list, labels, title):
    """
    Dibuja series temporales. Una figura por gráfico.
    """
    t = np.arange(len(series_list[0]))
    plt.figure()
    for s, lab in zip(series_list, labels):
        plt.plot(t, s, label=lab)
    plt.title(title)
    plt.xlabel("t (días)")
    plt.ylabel("personas")
    plt.legend()
    plt.tight_layout()
    plt.show()

def metricas_epidemia(I, R, V=None, N=None):
    """
    Métricas básicas de una simulación epidémica.
    - pico_I (valor máximo de I)
    - pico_% (si N se provee)
    - removidos_% (R_final/N si N se provee)
    - vacunados_% (V_final/N si V y N se proveen)
    """
    res = {"pico_I": float(np.max(I))}
    if N is not None and N > 0:
        res["pico_%"] = float(100*np.max(I)/N)
        res["removidos_%"] = float(100*R[-1]/N)
        if V is not None:
            res["vacunados_%"] = float(100*V[-1]/N)
    return res



## 1) Modelo SIR clásico
Ecuaciones con población fija $N=S_t+I_t+R_t$:
$$
\begin{aligned}
S_{t+1} &= S_t - \beta\,\frac{S_t I_t}{N},\\
I_{t+1} &= I_t + \beta\,\frac{S_t I_t}{N} - \gamma\, I_t,\\
R_{t+1} &= R_t + \gamma\, I_t.
\end{aligned}
$$

**Ejercicio**
1. Implementa `simulate_SIR(...)` con la firma dada abajo.  
2. Simula $160$ días con parámetros a simular y grafica $S,I,R$.  
3. Reporta: $R_0=\beta/\gamma$, **pico de $I$** y **ataque final** (si $N$ se provee).  
4. Describe en 2–4 bullets el comportamiento observado.


In [ ]:

def simulate_SIR(N, beta, gamma, I0=100, R0=0, days=160):
    """
    Simula el modelo SIR clásico.

    Parámetros
    ----------
    N : float
        Población total (constante).
    beta : float
        Tasa de contagio.
    gamma : float
        Tasa de recuperación.
    I0, R0 : float
        Condiciones iniciales (S0 = N - I0 - R0).
    days : int
        Número de días a simular.

    Returns
    -------
    S, I, R : np.ndarray
        Series temporales de longitud days+1.
    """
    # TODO: Implementar las actualizaciones S_{t+1}, I_{t+1}, R_{t+1}
    pass



**Parámetros a simular (SIR)**  
- $N=10^6$, $\beta=0.35$, $\gamma=1/7$, $I_0=100$, $R_0=0$, $T_{\text{sim}}=160$ días.


In [ ]:

# === Ejecuta después de implementar simulate_SIR ===
N=1_000_000; beta=0.35; gamma=1/7; days=160
S1, I1, R1 = simulate_SIR(N, beta, gamma, I0=100, R0=0, days=days)
graficar_trayectorias([S1, I1, R1], ["S","I","R"], "SIR — serie temporal")
m1 = metricas_epidemia(I1, R1, N=N)
print("R0 =", beta/gamma)
print("Métricas:", m1)



## 2) Modelo con Vacunación (SIRV)
Ecuaciones:
$$
\begin{aligned}
S_{t+1} &= S_t - \beta\,\frac{S_t I_t}{N} - v\, S_t,\\
I_{t+1} &= I_t + \beta\,\frac{S_t I_t}{N} - \gamma\, I_t,\\
R_{t+1} &= R_t + \gamma\, I_t,\\
V_{t+1} &= V_t + v\, S_t.
\end{aligned}
$$

**Ejercicio**
1. Implementa `simulate_SIRV(...)`.  
2. Simula con los mismos parámetros base del SIR y $v=0.002$. Grafica $S,I,R,V$.  
3. Reporta pico de $I$, porcentaje final de removidos y **% vacunados** al final.


In [ ]:

def simulate_SIRV(N, beta, gamma, v, I0=100, R0=0, V0=0, days=160):
    """
    Simula SIRV con vacunación continua a tasa v sobre susceptibles.

    Returns
    -------
    S, I, R, V : np.ndarray
    """
    # TODO: Implementar las actualizaciones S_{t+1}, I_{t+1}, R_{t+1}, V_{t+1}
    pass



**Parámetros a simular (SIRV)**  
- $N=10^6$, $\beta=0.35$, $\gamma=1/7$, $v=0.002$, $I_0=100$, $V_0=0$, $R_0=0$, $T_{\text{sim}}=160$.


In [ ]:

# === Ejecuta después de implementar simulate_SIRV ===
N=1_000_000; beta=0.35; gamma=1/7; days=160; v=0.002
S2, I2, R2, V2 = simulate_SIRV(N, beta, gamma, v, I0=100, R0=0, V0=0, days=days)
graficar_trayectorias([S2, I2, R2, V2], ["S","I","R","V"], "SIRV — serie temporal")
m2 = metricas_epidemia(I2, R2, V=V2, N=N)
print("Métricas:", m2)


## 3) Modelo con cuarentena por umbral (SIRVQ, instantáneo)
**Regla por casos:** si la proporción infectada $\frac{I_t}{N}$ es menor que $T$, no hay cuarentena; si es mayor o igual, se aísla una fracción $\rho$ de los susceptibles ese día (instantánea, **sin memoria de cohortes**).

$$
\text{Si } \frac{I_t}{N} < T:
\begin{cases}
S_{t+1} = S_t - \beta \dfrac{S_t I_t}{N} - v S_t, \\
I_{t+1} = I_t + \beta \dfrac{S_t I_t}{N} - \gamma I_t, \\
R_{t+1} = R_t + \gamma I_t, \\
V_{t+1} = V_t + v S_t, \\
Q_{t+1} = 0.
\end{cases}
\qquad
\text{Si } \frac{I_t}{N} \ge T:
\begin{cases}
S_{t+1} = S_t - \beta \dfrac{S_t I_t}{N} - v S_t - \rho S_t, \\
I_{t+1} = I_t + \beta \dfrac{S_t I_t}{N} - \gamma I_t, \\
R_{t+1} = R_t + \gamma I_t, \\
V_{t+1} = V_t + v S_t, \\
Q_{t+1} = \rho S_t.
\end{cases}
$$

**Ejercicio**
1. Implementa `simulate_SIRVQ_threshold_simple(...)`.  
2. Simula con $T=0.005$ (0.5%), $\rho=0.25$, $v=0.002$ y parámetros base del SIR.  
3. Grafica $S,I,R,V,Q$. Reporta: pico de $I$, ataque final, % vacunados y **días con umbral activo**.


In [ ]:

def simulate_SIRVQ_threshold_simple(N, beta, gamma, v, rho, T, I0=100, R0=0, V0=0, days=160):
    """
    SIRVQ con cuarentena INSTANTÁNEA por umbral en infectados (sin memoria).
    - Si I_t/N >= T: Q_{t+1} = rho * S_t y S_{t+1} resta rho*S_t
    - Si I_t/N <  T: Q_{t+1} = 0

    Returns
    -------
    S, I, R, V, Q : np.ndarray; active_days : int
    """
    # TODO: Implementar las actualizaciones por casos
    pass



**Parámetros a simular (SIRVQ)**  
- $N=10^6$, $\beta=0.35$, $\gamma=1/7$, $v=0.002$, $T=0.005$, $\rho=0.25$, $I_0=100$, $R_0=0$, $V_0=0$, $T_{\text{sim}}=160$ días.


In [ ]:

# === Ejecuta después de implementar simulate_SIRVQ_threshold_simple ===
N=1_000_000; beta=0.35; gamma=1/7; days=160; v=0.002; T=0.005; rho=0.25
S3, I3, R3, V3, Q3, active_days = simulate_SIRVQ_threshold_simple(N, beta, gamma, v, rho, T, I0=100, R0=0, V0=0, days=days)
graficar_trayectorias([S3, I3, R3, V3, Q3], ["S","I","R","V","Q"], "SIRVQ (umbral, instantáneo) — serie temporal")
m3 = metricas_epidemia(I3, R3, V=V3, N=N)
print("Métricas:", m3, "| Días con umbral activo:", active_days)
